# Mod 4 Project - Starter Notebook

This notebook has been provided to you so that you can make use of the following starter code to help with the trickier parts of preprocessing the Zillow dataset. 

The notebook contains a rough outline the general order you'll likely want to take in this project. You'll notice that most of the areas are left blank. This is so that it's more obvious exactly when you should make use of the starter code provided for preprocessing. 

**_NOTE:_** The number of empty cells are not meant to infer how much or how little code should be involved in any given step--we've just provided a few for your convenience. Add, delete, and change things around in this notebook as needed!

# Step 1: Load the Data/Filtering for Chosen Zipcodes

In [6]:
import pandas as pd

df= pd.read_csv('zillow_data.csv')
df_part_1=df.iloc[:, 7:]
df.head()



# df.iloc[:,-1]

,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,...,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04
0,84654,60657,Chicago,IL,Chicago,Cook,1,334200.0,335400.0,336500.0,...,1005500,1007500,1007800,1009600,1013300,1018700,1024400,1030700,1033800,1030600
1,90668,75070,McKinney,TX,Dallas-Fort Worth,Collin,2,235700.0,236900.0,236700.0,...,308000,310000,312500,314100,315000,316600,318100,319600,321100,321800
2,91982,77494,Katy,TX,Houston,Harris,3,210400.0,212200.0,212200.0,...,321000,320600,320200,320400,320800,321200,321200,323000,326900,329900
3,84616,60614,Chicago,IL,Chicago,Cook,4,498100.0,500900.0,503100.0,...,1289800,1287700,1287400,1291500,1296600,1299000,1302700,1306400,1308500,1307000
4,93144,79936,El Paso,TX,El Paso,El Paso,5,77300.0,77300.0,77300.0,...,119100,119400,120000,120300,120300,120300,120300,120500,121000,121500


In [7]:
df['AvgHomePrice']= df.mean(skipna=True, axis=1)

In [8]:
df['ROI']= (df.iloc[:,-2]-df.iloc[:,7])/df.iloc[:,7]

In [9]:
df.head()

,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,...,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04,AvgHomePrice,ROI
0,84654,60657,Chicago,IL,Chicago,Cook,1,334200.0,335400.0,336500.0,...,1007800,1009600,1013300,1018700,1024400,1030700,1033800,1030600,736192.955224,2.083782
1,90668,75070,McKinney,TX,Dallas-Fort Worth,Collin,2,235700.0,236900.0,236700.0,...,312500,314100,315000,316600,318100,319600,321100,321800,217815.447761,0.365295
2,91982,77494,Katy,TX,Houston,Harris,3,210400.0,212200.0,212200.0,...,320200,320400,320800,321200,321200,323000,326900,329900,259808.876866,0.567966
3,84616,60614,Chicago,IL,Chicago,Cook,4,498100.0,500900.0,503100.0,...,1287400,1291500,1296600,1299000,1302700,1306400,1308500,1307000,963776.619403,1.623971
4,93144,79936,El Paso,TX,El Paso,El Paso,5,77300.0,77300.0,77300.0,...,120000,120300,120300,120300,120300,120500,121000,121500,101380.914179,0.571798


In [21]:
top_10_ROI= df[['RegionID', 'RegionName', 'City', 'State', 'Metro', 'CountyName', 'SizeRank', 'AvgHomePrice', 'ROI']].sort_values('ROI', ascending= False).head(10)

In [22]:
top_10_ROI

,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,AvgHomePrice,ROI
117,62022,11211,New York,NY,New York,Kings,118,5.887785e+05,11.189940
1155,62033,11222,New York,NY,New York,Kings,1156,6.331605e+05,10.535523
475,62027,11216,New York,NY,New York,Kings,476,5.660419e+05,9.942505
191,60639,7302,Jersey City,NJ,New York,Hudson,192,5.901673e+05,9.403061
106,62026,11215,New York,NY,New York,Kings,107,1.063203e+06,8.941958
13864,62320,11975,Wainscott,NY,New York,Suffolk,13865,1.556934e+06,8.922175
11728,62281,11930,Amagansett,NY,New York,Suffolk,11729,1.446869e+06,8.564860
14217,93399,80449,Hartsel,CO,Denver,Park,14218,1.448611e+05,7.923333
1790,96127,90291,Los Angeles,CA,Los Angeles-Long Beach-Anaheim,Los Angeles,1791,9.179616e+05,7.778489
258,66125,20001,Washington,DC,Washington,District of Columbia,259,3.809570e+05,7.622826


In [28]:
top_10_AvgHomePrice= df[['RegionID', 'RegionName', 'City', 'State', 'Metro', 'CountyName', 'SizeRank', 'AvgHomePrice', 'ROI']].sort_values('AvgHomePrice', ascending= False).head(25)

In [29]:
top_10_AvgHomePrice

,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,AvgHomePrice,ROI
272,61635,10021,New York,NY,New York,New York,273,1.263432e+07,NaN
20,61625,10011,New York,NY,New York,New York,21,7.613538e+06,NaN
508,61628,10014,New York,NY,New York,New York,509,6.720111e+06,NaN
21,61703,10128,New York,NY,New York,New York,22,5.028777e+06,1.009030
10237,97518,94027,Atherton,CA,San Francisco,San Mateo,10238,3.448847e+06,4.763653
7596,93816,81611,Aspen,CO,Glenwood Springs,Pitkin,7597,3.112578e+06,2.303028
4816,96086,90210,Beverly Hills,CA,Los Angeles-Long Beach-Anaheim,Los Angeles,4817,2.759459e+06,4.866358
4723,72636,33480,Palm Beach,FL,Miami-Fort Lauderdale,Palm Beach,4724,2.605421e+06,3.889608
2026,97577,94123,San Francisco,CA,San Francisco,San Francisco,2027,2.602249e+06,4.514427
742,97569,94115,San Francisco,CA,San Francisco,San Francisco,743,2.372893e+06,4.239924


In [ ]:
# if df.iloc[:,7]='Nan':
#     counter=0
#     counter+=1
#     df.iloc[:,7+counter]

# Add in code to make a for loop that essentially uses the first value that is not NaN to then do ROI for.

In [ ]:
df.sort_values('ROI', ascending=False)

# Step 2: Data Preprocessing

In [ ]:
def get_datetimes(df):
    return pd.to_datetime(df.columns.values[1:], format='%Y-%m')

In [ ]:
get_datetimes(df)

# Step 3: EDA and Visualization

In [ ]:
font = {'family' : 'normal',
        'weight' : 'bold',
        'size'   : 22}

matplotlib.rc('font', **font)

# Step 4: Reshape from Wide to Long Format

In [ ]:
def melt_data(df):
    melted = pd.melt(df, id_vars=['RegionName', 'City', 'State', 'Metro', 'CountyName'], var_name='time')
    melted['time'] = pd.to_datetime(melted['time'], infer_datetime_format=True)
    melted = melted.dropna(subset=['value'])
    return melted.groupby('time').aggregate({'value':'mean'})

# Step 5: ARIMA Modeling

# Step 6: Interpreting Results